# Hanukkah of Data 2022

The process of my solution to the [Hanukkah of Data 2022](https://hanukkah.bluebird.sh/5783/).

## Table of contents

1. [Day 1](#day1)
2. [Day 2](#day2)
3. [Day 3](#day3)
4. [Day 4](#day4)
5. [Day 5](#day5)
6. [Day 6](#day6)
7. [Day 7](#day7)
8. [Day 8](#day8)

In [1]:
import pandas as pd
import re

In [2]:
customers = pd.read_csv("noahs-customers.csv")
customers["birthdate"] = pd.to_datetime(customers["birthdate"])
customers.head()

,customerid,name,address,citystatezip,birthdate,phone
0,1001,Jack Quinn,201 E Park St,"Los Angeles, CA 91343",1960-05-14,805-287-8515
1,1002,David Powell,224C Tysens Ln,"Staten Island, NY 10306",1978-04-04,516-768-1652
2,1003,Carrie Green,1608 W 53rd Way,"Tampa, FL 33614",1969-01-21,727-209-0470
3,1004,Steven Miller,178½ E 7th St,"Manhattan, NY 10009",1953-08-17,607-941-9563
4,1005,Christine Powers,270 W 242nd St,"Bronx, NY 10463",1983-06-06,212-759-9043


In [3]:
orders = pd.read_csv("noahs-orders.csv")
orders["ordered"] = pd.to_datetime(orders["ordered"])
orders["shipped"] = pd.to_datetime(orders["shipped"])
orders.head()

,orderid,customerid,ordered,shipped,items,total
0,1001,4308,2017-01-31 00:32:19,2017-01-31 07:15:00,NaN,25.52
1,1002,11683,2017-01-31 00:58:31,2017-01-31 18:00:00,NaN,35.33
2,1003,5676,2017-01-31 01:34:40,2017-01-31 09:00:00,NaN,30.79
3,1004,3097,2017-01-31 02:31:24,2017-01-31 19:45:00,NaN,77.60
4,1005,10374,2017-01-31 02:46:09,2017-01-31 14:45:00,NaN,109.04


In [4]:
orders_items = pd.read_csv("noahs-orders_items.csv")
orders_items.head()

,orderid,sku,qty,unit_price
0,1001,COL0820,1,25.52
1,1002,TOY8907,1,12.92
2,1002,KIT5813,1,7.99
3,1002,KIT3981,2,7.21
4,1003,KIT7098,1,12.53


In [5]:
products = pd.read_csv("noahs-products.csv")
products.head()

,sku,desc,wholesale_cost
0,DLI0002,Smoked Whitefish Sandwich,9.33
1,PET0005,"Vegan Cat Food, Turkey & Chicken",4.35
2,HOM0018,Power Radio (red),21.81
3,KIT0034,Azure Ladle,2.81
4,PET0041,"Gluten-free Cat Food, Pumpkin & Pumpkin",4.60


## Day 1  <a name="day1"></a>

According to the challenge, **"you could get a number that spelled something using the letters printed on the phone buttons: like 2 has “ABC”, and 3 “DEF”, etc."**
To find the letter assosiate with number. I use [E.161](https://en.wikipedia.org/wiki/E.161) system as the reference.

Then I use the `name_number` function to check if when convert name to number, the number is match with their phone number or not.

In [6]:
# Function to find the customer who when change their last name to letter, it equal to their phone number
def name_number(name, phone):
    name_to_num = ""
    digit2 = ["a", "b", "c", "2"]
    digit3 = ["d", "e", "f", "3"]
    digit4 = ["g", "h", "i", "4"]
    digit5 = ["j", "k", "l", "5"]
    digit6 = ["m", "n", "o", "6"]
    digit7 = ["p", "q", "r", "s", "7"]
    digit8 = ["t", "u", "v", "8"]
    digit9 = ["w", "x", "y", "z", "9"]
    digit_list = [digit2, digit3, digit4, digit5, digit6, digit7, digit8, digit9]
    name_list = name.split(" ")
    phone_short = phone.replace("-","")
    for character in name_list[1]:
        for d_ls in digit_list:
            if character.lower() in d_ls:
                name_to_num += d_ls[-1]
    if name_to_num == phone_short:
        print(phone)

#Test
#x = "Christine Powers"
#name_number(x, "212-759-9043")

In [7]:
# Return the phone number of customer with name to number match with phone number
_ = customers.apply(lambda x: name_number(x["name"], x["phone"]), axis=1)

488-836-2374


## Day 2 <a name="day2"></a>

**"‘JD’ is the initials of the contractor."**

First, I identify the customer with the initials JD.

In [8]:
def find_jd(name):
    name_list = name.split(" ")
    if name_list[0][0].upper() == "J" and name_list[-1][0].upper() == "D":
        return True
    else:
        return False

In [9]:
customers["JD"] = customers["name"].apply(find_jd)
customer_JD = customers[customers["JD"]==True]
customer_JD.head()

,customerid,name,address,citystatezip,birthdate,phone,JD
443,1444,Jennifer Davis,4663 Park Ave S,"Brooklyn, NY 11232",1940-07-29,315-356-9127,True
513,1514,Joshua Downs,144 Park Rd N,"McAllen, TX 78557",1992-05-22,682-474-3747,True
610,1611,Jacob Davis,478B E 91st St,"Manhattan, NY 10128",1961-06-02,516-521-7244,True
741,1742,Jonathan Davis,862 Morris Park Ave,"Bronx, NY 10462",1947-05-08,585-510-8718,True
753,1754,Jennifer Diaz,135B Rogers Ave,"Brooklyn, NY 11216",1982-05-20,838-499-3728,True


Then from the clue **‘2017’ is the year the item was brought in**. Find the order from the customer with initial JD that order in 2017.

In [10]:
customer_JD_m = pd.merge(customer_JD, orders, how="inner", on="customerid")[["customerid","name","phone","orderid","ordered"]]
customer_JD_m = pd.merge(customer_JD_m, orders_items, how="inner", on="orderid")
customer_JD_m = pd.merge(customer_JD_m, products, how="inner", on="sku")
customer_suspect = customer_JD_m[customer_JD_m["ordered"].dt.year == 2017][["name","phone","ordered","desc"]]
customer_suspect

,name,phone,ordered,desc
0,Jacob Davis,516-521-7244,2017-02-03 11:34:48,Mechanical Blocks
2,Jacob Davis,516-521-7244,2017-02-03 11:34:48,Noah's Jersey (white)
4,James Davidson,516-515-6965,2017-11-26 19:00:40,Noah's Jersey (white)
5,Julie Melissa Duffy,347-716-8024,2017-10-02 13:57:06,Noah's Jersey (white)
8,Jacob Davis,516-521-7244,2017-03-03 10:23:46,Manual Onion Cup
...,...,...,...,...
2510,Jeff Diaz,518-223-6561,2017-06-08 08:13:44,Power Radio (magenta)
2513,Joshua Dickerson,315-629-8623,2017-05-21 12:34:22,"Dry Adult Cat Food, Tuna & Duck"
2514,Joshua Dickerson,315-629-8623,2017-08-01 08:18:36,"Vegan Cat Food, Turkey & Duck"
2516,Joshua Dickerson,315-629-8623,2017-09-19 08:07:25,Spatula


Lastly, from the clue **they usually talked about the project over coffee and bagels**. Find the customer that order bagel or coffee.

In [11]:
def coffee_bagel(desc):
    desc_list = desc.lower().split(" ")
    if "coffee" in desc_list or "bagel" in desc_list:
        return True
    else:
        return False

In [12]:
customer_suspect["coffee_bagel"] = customer_suspect["desc"].apply(coffee_bagel)
customer_suspect_coba = customer_suspect[customer_suspect["coffee_bagel"]==True]
customer_suspect_coba

,name,phone,ordered,desc,coffee_bagel
1695,James Dixon,212-277-2382,2017-10-30 11:21:15,Mechanical Coffee Spatula,True
1732,Jeremy Davis,212-771-8924,2017-04-05 12:49:41,Vintage Coffee Strainer,True
1733,Julie Melissa Duffy,347-716-8024,2017-10-29 13:40:05,Vintage Coffee Strainer,True
1734,Jeremy Davis,212-771-8924,2017-04-05 12:49:41,Sesame Bagel,True
2427,Javier Diaz,838-264-0667,2017-11-13 20:06:42,Handmade Coffee Spatula,True


From the list, the only customer that order the food, not utensil for coffee (or bagel) is Jeremy Davis, phone number 838-264-0667.

## Day 3 <a name="day3"></a>

From the clue **He said that he was naturally assertive because he was a Aries born in the year of the Dog**, first find the customer that Aries born [(March 20 – April 20)](https://en.wikipedia.org/wiki/Aries_(astrology)) and year of [dog](https://en.wikipedia.org/wiki/Dog_(zodiac)).

In [13]:
dog_years = [1922, 1934, 1946, 1958, 1970, 1982, 1994, 2006, 2018]

customer_spider = customers[customers["birthdate"].dt.year.isin(dog_years)]
customer_spider = customer_spider[((customer_spider["birthdate"].dt.month==3) &  \
                                  (customer_spider["birthdate"].dt.day >= 20) | \
                                  (customer_spider["birthdate"].dt.month==4) & \
                                  (customer_spider["birthdate"].dt.day <=20))]
customer_spider

,customerid,name,address,citystatezip,birthdate,phone,JD
77,1078,Eric Brown,3186A Barnes Ave,"Bronx, NY 10467",1982-04-16,838-862-8138,False
151,1152,April Reynolds,3119 Park Rd SW,"Omaha, NE 68102",1958-04-15,531-313-8952,False
183,1184,Jeffrey White,459 W 81st Pl,"San Juan, PR 00913",1994-04-12,521-945-0864,False
253,1254,Tristan Warner,135-35 62nd Ave,"Flushing, NY 11367",1970-03-20,212-234-4766,False
270,1271,Jay Vasquez Jr.,1872 E Tremont Ave,"Bronx, NY 10460",1970-04-15,631-825-8312,False
...,...,...,...,...,...,...,...
10624,11624,Lisa Williams,123-61 Metropolitan Ave,"Kew Gardens, NY 11415",1958-04-06,716-987-7331,False
10692,11692,Ashley Abbott,158-21 102nd St,"Howard Beach, NY 11414",1970-03-23,716-824-1678,False
10808,11808,Christina Sparks,3486 Park Pl S,"Kearney, MO 64060",1970-04-12,314-876-7301,False
10813,11813,Bethany Watson,3 Sniffen Ct,"Manhattan, NY 10016",1970-04-03,680-798-9786,False


From clue **"so I gave it to this guy who lived in my neighborhood."**. Let's check the contractor neighborhood and find the person from the list that live in the same neighborhood as the contractor.

In [14]:
customers[customers["phone"]=="212-771-8924"]

,customerid,name,address,citystatezip,birthdate,phone,JD
3163,4164,Jeremy Davis,134-10 Foch Blvd,"South Ozone Park, NY 11420",1995-07-26,212-771-8924,True


In [15]:
def address_neighbor(address, looking_address):
    address_list = address.lower().split(" ")
    if looking_address in address_list:
        return True
    else:
        return False

In [16]:
customer_spider["neighbor"] = customer_spider["citystatezip"].apply(address_neighbor, args=("ozone",))
customer_spider[customer_spider["neighbor"]==True]


,customerid,name,address,citystatezip,birthdate,phone,JD,neighbor
1273,2274,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397,False,True


## Day 4 <a name="day4"></a>

Since there was no column that directly identify which product is pastries, the closest identification seems to be letters in SKU. To find the letter that relate to pastries, I use bagel to check the SKU because it is the only pastries that I know Noah definitely sold (from day 2 challenge).

After checking, I found that the bagel has SKU contain BKY, which make sense since it might come from the word bakery.

In [17]:
product_list = set(products["desc"])
for p in product_list:
    p_list = p.lower().split(" ")
    if "bagel" in p_list:
        print(p)

Caraway Bagel
Sesame Bagel


In [18]:
products[(products["desc"]=="Caraway Bagel") | (products["desc"]=="Sesame Bagel")]

,sku,desc,wholesale_cost
467,BKY4234,Caraway Bagel,6.23
649,BKY5887,Sesame Bagel,6.38


Then from clue **"she liked to get up before dawn"** and **"came over at 5am "** I know that she must order the pastries early in the morning and must be before 5am. So I should find the order with ordered time and shipped time are early in the morning and before 5 am.

In [19]:
order_m = pd.merge(orders, orders_items, how="inner", on="orderid")
order_m = pd.merge(order_m, products, how="inner", on="sku")
order_m.head()

,orderid,customerid,ordered,shipped,items,total,sku,qty,unit_price,desc,wholesale_cost
0,1001,4308,2017-01-31 00:32:19,2017-01-31 07:15:00,NaN,25.52,COL0820,1,25.52,Noah's Lunchbox (blue),20.96
1,1137,1968,2017-02-01 13:03:11,2017-02-01 13:03:11,NaN,26.26,COL0820,1,26.26,Noah's Lunchbox (blue),20.96
2,1994,6225,2017-02-09 21:04:57,2017-02-12 15:00:00,NaN,234.76,COL0820,2,23.58,Noah's Lunchbox (blue),20.96
3,3274,8637,2017-02-23 10:27:49,2017-02-23 10:27:49,NaN,234.45,COL0820,1,29.05,Noah's Lunchbox (blue),20.96
4,3735,7527,2017-02-28 09:03:43,2017-02-28 09:03:43,NaN,27.56,COL0820,1,27.56,Noah's Lunchbox (blue),20.96


In [20]:
# Filter only order with product SKU start with BKY
order_m["sku_cat"] = order_m["sku"].str[:3]
order_bky = order_m[order_m["sku_cat"]=="BKY"]
order_bky.head()

,orderid,customerid,ordered,shipped,items,total,sku,qty,unit_price,desc,wholesale_cost,sku_cat
12098,1018,3869,2017-01-31 10:00:26,2017-01-31 19:30:00,NaN,11.01,BKY8673,1,5.89,Raspberry Linzer Cookie,4.97,BKY
12099,1201,9205,2017-02-02 07:47:19,2017-02-02 09:30:00,NaN,13.04,BKY8673,1,6.02,Raspberry Linzer Cookie,4.97,BKY
12100,1229,10929,2017-02-02 11:54:39,2017-02-02 11:54:39,NaN,11.01,BKY8673,1,5.97,Raspberry Linzer Cookie,4.97,BKY
12101,1455,6900,2017-02-04 11:28:17,2017-02-04 13:15:00,NaN,115.35,BKY8673,1,6.61,Raspberry Linzer Cookie,4.97,BKY
12102,1825,6321,2017-02-08 11:03:35,2017-02-08 11:03:35,NaN,66.58,BKY8673,1,6.67,Raspberry Linzer Cookie,4.97,BKY


In [21]:
dawn_order = order_bky[(order_bky["ordered"].dt.hour >= 3) & (order_bky["ordered"].dt.hour < 5) & \
                       (order_bky["shipped"].dt.hour >= 3) & (order_bky["shipped"].dt.hour < 5) ]
dawn_order

,orderid,customerid,ordered,shipped,items,total,sku,qty,unit_price,desc,wholesale_cost,sku_cat
28894,201780,5375,2022-08-03 04:40:13,2022-08-03 04:40:13,NaN,5.39,BKY3104,1,5.39,Caraway Puff,4.36,BKY
53717,57114,10300,2018-08-16 03:45:20,2018-08-16 03:45:20,NaN,145.63,BKY4940,1,6.66,Raspberry Hamentash,5.31,BKY
99644,104231,5676,2019-11-27 03:27:15,2019-11-27 04:15:00,NaN,13.65,BKY6845,1,5.97,Poppyseed Babka,5.44,BKY
111567,26173,5105,2017-10-11 04:58:38,2017-10-11 04:58:38,NaN,389.65,BKY5887,1,7.89,Sesame Bagel,6.38,BKY
225252,93583,5375,2019-08-14 04:09:04,2019-08-14 04:09:04,NaN,5.60,BKY6881,1,5.60,Poppyseed Rugelach,5.35,BKY
239541,43673,5375,2018-04-04 04:25:24,2018-04-04 04:25:24,NaN,6.07,BKY0403,1,6.07,Poppyseed Linzer Cookie,4.87,BKY
245437,148755,8790,2021-02-17 03:20:45,2021-02-17 03:20:45,NaN,5.76,BKY3437,1,5.76,Raspberry Mandelbrot,4.89,BKY
260759,199892,5375,2022-07-15 04:38:38,2022-07-15 04:38:38,NaN,14.50,BKY7160,2,7.25,Caraway Twist,6.15,BKY
267266,161847,5375,2021-06-28 04:10:43,2021-06-28 04:10:43,NaN,12.36,BKY4022,2,6.18,Sesame Bialy,5.86,BKY
267323,196511,5375,2022-06-11 04:44:42,2022-06-11 04:44:42,NaN,7.72,BKY4022,1,7.72,Sesame Bialy,5.86,BKY


Among the early morning order, the customer ID 5375 is the most standout customer with 10 bakery's orders early in the morning. This resemble with the clue **"she liked to get up before dawn and claim the first pastries that came out of the oven."**

In [22]:
dawn_order.groupby("customerid").count().reset_index()[["customerid","orderid"]]

,customerid,orderid
0,5105,1
1,5375,10
2,5676,1
3,8790,1
4,10300,1
5,10543,1


In [23]:
customers[customers["customerid"]==5375]

,customerid,name,address,citystatezip,birthdate,phone,JD
4374,5375,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,False


## Day 5 <a name="day5"></a>

From the clue, **"she only had ten or eleven cats and they were getting quite old and had cataracts"** and she is likely to buy a lot of cat food from Noah.

So, I explore the list of the products and found that Noah sold Senior Cat Food and list the SKU of those products.

In [24]:
product_list = set(products["desc"])
for p in product_list:
    p_list = p.lower().split(" ")
    if "senior" in p_list and "cat" in p_list:
        print(p)
        break

Vegan Senior Cat Food, Duck & Pumpkin


In [25]:
def cat_food(desc):
    desc_list = desc.lower().split(" ")
    if "senior" in desc_list and "cat" in desc_list:
        return True
    else:
        return False

In [26]:
product_cat = products.copy()
product_cat["cat_food"] = product_cat["desc"].apply(cat_food)
product_cat = product_cat[product_cat["cat_food"]==True]
product_cat.head()

,sku,desc,wholesale_cost,cat_food
11,PET0096,"Wet Senior Cat Food, Turkey & Shrimp",3.11,True
42,PET0304,"Gluten-free Senior Cat Food, Chicken & Turkey",4.24,True
96,PET0925,Senior Cat Food,3.84,True
100,PET0977,"Vegan Senior Cat Food, Chicken & Beef",4.36,True
115,PET1090,"Vegan Senior Cat Food, Tuna & Duck",4.39,True


There is also a clue that she live in **"Queens Village"**

In [27]:
customer_freec = customers.copy()
customer_freec["queens"] = customer_freec["citystatezip"].apply(address_neighbor, args=("queens",))
customer_freec = customer_freec[customer_freec["queens"]==True]
customer_freec.head()

,customerid,name,address,citystatezip,birthdate,phone,JD,queens
111,1112,Barbara Krystal Reeves,101-24 212th St,"Queens Village, NY 11429",1967-06-25,680-765-4131,False,True
291,1292,Johnathan Garcia,219-43 113th Dr,"Queens Village, NY 11429",1997-08-14,516-288-6132,False,True
365,1366,Andrea Sullivan,220-54 111th Ave,"Queens Village, NY 11429",1961-11-07,347-664-7228,False,True
417,1418,Allison Coleman,102-5 Springfield Blvd,"Queens Village, NY 11429",1977-04-01,212-708-9172,False,True
677,1678,Anita Becker,215-66 99th Ave,"Queens Village, NY 11429",1963-03-16,631-924-9714,False,True


Combine 2 clues together and find the customers that purchase the most senior cat food.

In [28]:
customer_freec_m = pd.merge(customer_freec, orders, how="inner", on="customerid")
customer_freec_m = pd.merge(customer_freec_m, orders_items, how="inner", on="orderid")
customer_freec_m.head()

,customerid,name,address,citystatezip,birthdate,phone,JD,queens,orderid,ordered,shipped,items,total,sku,qty,unit_price
0,1418,Allison Coleman,102-5 Springfield Blvd,"Queens Village, NY 11429",1977-04-01,212-708-9172,False,True,176756,2021-11-23 12:58:22,2021-11-23 13:30:00,NaN,2.30,KIT0098,1,2.30
1,3335,Jose Cook,225-24 Murdock Ave,"Queens Village, NY 11429",1950-03-27,516-745-9461,False,True,176835,2021-11-24 10:29:46,2021-11-24 10:29:46,NaN,22.57,COL7385,1,19.42
2,3335,Jose Cook,225-24 Murdock Ave,"Queens Village, NY 11429",1950-03-27,516-745-9461,False,True,176835,2021-11-24 10:29:46,2021-11-24 10:29:46,NaN,22.57,PET1537,1,3.15
3,3356,Kimberly Hernandez,226-41 Murdock Ave,"Queens Village, NY 11429",1984-06-15,516-366-7376,False,True,22269,2017-09-02 17:55:18,2017-09-07 11:00:00,NaN,5.58,TOY7876,1,5.58
4,3358,Kathy Jane Floyd,103-20 215th St,"Queens Village, NY 11429",1998-10-27,838-923-4863,False,True,9837,2017-04-29 22:06:18,2017-05-02 10:45:00,NaN,123.45,HOM0284,1,52.38


In [29]:
cat_food_list = list(product_cat["sku"])
customer_catfood = customer_freec_m[customer_freec_m["sku"].isin(cat_food_list)]
customer_catfood.groupby("phone").count().reset_index()[["phone","customerid"]]

,phone,customerid
0,315-492-7411,17
1,607-836-2966,2


## Day 6 <a name="day6"></a>

From the clue **"In fact I like to tease her that Noah actually loses money whenever she comes in the store."**,  I find the customer that has the most number of loss order (total wholesale cost > total order price)

In [30]:
order_product = pd.merge(orders_items, products, how="inner", on="sku")
order_product.head()

,orderid,sku,qty,unit_price,desc,wholesale_cost
0,1001,COL0820,1,25.52,Noah's Lunchbox (blue),20.96
1,1137,COL0820,1,26.26,Noah's Lunchbox (blue),20.96
2,1994,COL0820,2,23.58,Noah's Lunchbox (blue),20.96
3,3274,COL0820,1,29.05,Noah's Lunchbox (blue),20.96
4,3735,COL0820,1,27.56,Noah's Lunchbox (blue),20.96


In [31]:
order_product["tot_price"] = order_product["qty"] * order_product["unit_price"]
order_product["tot_wholesale"] = order_product["qty"] * order_product["wholesale_cost"]

sum_order = order_product[["orderid", "tot_price", "tot_wholesale"]].groupby("orderid").sum().reset_index()
loss_order = sum_order[sum_order["tot_wholesale"]>=sum_order["tot_price"]]
loss_order.head()

,orderid,tot_price,tot_wholesale
596,1597,4.24,4.32
1027,2028,4.00,4.17
1074,2075,7.11,7.51
1566,2567,2.67,2.68
1577,2578,7.99,8.48


In [32]:
loss_order_cus = pd.merge(loss_order, orders, how="inner", on="orderid")
loss_order_cus = pd.merge(loss_order_cus, customers, how="inner", on="customerid")

loss_order_cus.groupby("phone").count().reset_index()[["phone", "orderid"]].sort_values(by="orderid", ascending=False).head()

,phone,orderid
662,914-868-0316,15
105,332-668-0158,3
275,516-986-1842,3
258,516-938-2079,3
639,914-548-7137,3


## Day 7 <a name="day7"></a>

From clue **" He said ‘I got almost exactly the same thing!’ We laughed about it and wound up swapping items because he had wanted the color I got."** Find all item that Emily Randolph with phone number 914-868-0316 (day 6 answer) bought.

In [33]:
cus_day6 = customers[customers["phone"]=="914-868-0316"]
cus_day6

,customerid,name,address,citystatezip,birthdate,phone,JD
7341,8342,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316,False


In [34]:
cus_day6_order = pd.merge(cus_day6, orders, how="inner", on="customerid")
cus_day6_order = pd.merge(cus_day6_order, orders_items, how="inner", on="orderid")
cus_day6_order = pd.merge(cus_day6_order, products, how="inner", on="sku")
cus_day6_order[["name","orderid","ordered","shipped","sku", "desc"]]

,name,orderid,ordered,shipped,sku,desc
0,Emily Randolph,12624,2017-05-28 09:15:06,2017-05-28 11:00:00,HOM7105,Handmade Toaster (blue)
1,Emily Randolph,33563,2017-12-24 17:05:45,2017-12-24 17:05:45,KIT0100,Super Coconut Fork
2,Emily Randolph,75257,2019-02-11 18:52:45,2019-02-11 18:52:45,TOY9308,Disney Dreidel
3,Emily Randolph,86371,2019-06-01 12:50:16,2019-06-01 12:50:16,HOM5139,Electric Machine (amber)
4,Emily Randolph,87599,2019-06-13 15:56:36,2019-06-13 15:56:36,PET6786,Senior Mouse Food
5,Emily Randolph,87599,2019-06-13 15:56:36,2019-06-13 15:56:36,PET5196,"Gluten-free Senior Cat Food, Salmon & Pumpkin"
6,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,HOM1216,Manual Mixer (orange)
7,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,KIT4742,Manual Coffee Bowl
8,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,HOM5127,Electric Toaster (blue)
9,Emily Randolph,96220,2019-09-09 16:30:05,2019-09-09 17:45:00,COL4384,Noah's Poster (white)


From the list, there multiple items with colour that Emily bought. (Though from the list, dropping Handmade Toaster and switch it with another person is unlikely, but not impossible.)

Then, split the item's name and the colour to be another column.

In [35]:
def find_colour(desc):
    if re.search("\([^\s-]*\)", desc):
        return True
    else:
        return False

In [36]:
cus_day6_order["colour"] = cus_day6_order["desc"].apply(find_colour)
cus_day6_colour = cus_day6_order.copy()
cus_day6_colour = cus_day6_colour[cus_day6_colour["colour"]==True]
cus_day6_colour["shipped_date"] = cus_day6_colour["shipped"].dt.date
cus_day6_colour = cus_day6_colour[["name","orderid","ordered","shipped","shipped_date","sku", "desc"]]
cus_day6_colour

,name,orderid,ordered,shipped,shipped_date,sku,desc
0,Emily Randolph,12624,2017-05-28 09:15:06,2017-05-28 11:00:00,2017-05-28,HOM7105,Handmade Toaster (blue)
3,Emily Randolph,86371,2019-06-01 12:50:16,2019-06-01 12:50:16,2019-06-01,HOM5139,Electric Machine (amber)
6,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,2019-08-18,HOM1216,Manual Mixer (orange)
8,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,2019-08-18,HOM5127,Electric Toaster (blue)
9,Emily Randolph,96220,2019-09-09 16:30:05,2019-09-09 17:45:00,2019-09-09,COL4384,Noah's Poster (white)
21,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,2021-04-17,COL4001,Noah's Jewelry (yellow)
22,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,2021-04-17,HOM5838,Electric Crockpot (purple)
25,Emily Randolph,200947,2022-07-26 08:18:05,2022-07-26 16:00:00,2022-07-26,COL3831,Noah's Jewelry (orange)


In [37]:
def get_item_name(desc):
    desc_list = desc.split(" ")
    return pd.Series([(" ").join(desc_list[:-1]), desc_list[-1]])

In [38]:
cus_day6_colour[["desc_wo_colour", "desc_colour"]] = cus_day6_colour["desc"].apply(get_item_name)
cus_day6_colour = cus_day6_colour.rename(columns={"ordered":"ordered_cd6", 
                                "shipped":"shipped_cd6",
                                "sku":"sku_cd6",
                                "desc":"desc_cd6",
                                 "desc_colour":"desc_colour_cd6"})
cus_day6_colour

,name,orderid,ordered_cd6,shipped_cd6,shipped_date,sku_cd6,desc_cd6,desc_wo_colour,desc_colour_cd6
0,Emily Randolph,12624,2017-05-28 09:15:06,2017-05-28 11:00:00,2017-05-28,HOM7105,Handmade Toaster (blue),Handmade Toaster,(blue)
3,Emily Randolph,86371,2019-06-01 12:50:16,2019-06-01 12:50:16,2019-06-01,HOM5139,Electric Machine (amber),Electric Machine,(amber)
6,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,2019-08-18,HOM1216,Manual Mixer (orange),Manual Mixer,(orange)
8,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,2019-08-18,HOM5127,Electric Toaster (blue),Electric Toaster,(blue)
9,Emily Randolph,96220,2019-09-09 16:30:05,2019-09-09 17:45:00,2019-09-09,COL4384,Noah's Poster (white),Noah's Poster,(white)
21,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,2021-04-17,COL4001,Noah's Jewelry (yellow),Noah's Jewelry,(yellow)
22,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,2021-04-17,HOM5838,Electric Crockpot (purple),Electric Crockpot,(purple)
25,Emily Randolph,200947,2022-07-26 08:18:05,2022-07-26 16:00:00,2022-07-26,COL3831,Noah's Jewelry (orange),Noah's Jewelry,(orange)


Use the list of products that Emily bought to find other order with the same item but different colour.

In [39]:
all_p_colour = products.copy()
all_p_colour["colour"] = all_p_colour["desc"].apply(find_colour)
all_p_colour = all_p_colour[all_p_colour["colour"]==True]
all_p_colour [["desc_wo_colour", "desc_colour"]] = all_p_colour ["desc"].apply(get_item_name)
all_p_colour = pd.merge(all_p_colour, orders_items, how="inner", on="sku")
all_p_colour = pd.merge(all_p_colour, orders, how="inner", on="orderid")
all_p_colour["shipped_date"] = all_p_colour["shipped"].dt.date

all_p_colour = all_p_colour[["orderid","customerid","sku", "desc","desc_wo_colour","desc_colour", "shipped", "shipped_date"]]
all_p_colour 

,orderid,customerid,sku,desc,desc_wo_colour,desc_colour,shipped,shipped_date
0,1198,3241,HOM0018,Power Radio (red),Power Radio,(red),2017-02-02 15:15:00,2017-02-02
1,1504,9901,HOM0018,Power Radio (red),Power Radio,(red),2017-02-04 18:55:25,2017-02-04
2,1504,9901,HOM1734,Mechanical Crockpot (magenta),Mechanical Crockpot,(magenta),2017-02-04 18:55:25,2017-02-04
3,2139,6298,HOM0018,Power Radio (red),Power Radio,(red),2017-02-11 17:45:00,2017-02-11
4,2366,7360,HOM0018,Power Radio (red),Power Radio,(red),2017-02-13 15:12:54,2017-02-13
...,...,...,...,...,...,...,...,...
70749,210112,6366,HOM9988,Mini Crockpot (purple),Mini Crockpot,(purple),2022-10-27 12:39:25,2022-10-27
70750,210278,7389,HOM9988,Mini Crockpot (purple),Mini Crockpot,(purple),2022-10-28 18:30:27,2022-10-28
70751,211929,3842,HOM9988,Mini Crockpot (purple),Mini Crockpot,(purple),2022-11-14 11:27:48,2022-11-14
70752,212676,5744,HOM9988,Mini Crockpot (purple),Mini Crockpot,(purple),2022-11-21 10:57:38,2022-11-21


Find the order with same item, same date, but different colour.

In [40]:
same_item_date = pd.merge(all_p_colour, cus_day6_colour, how="inner", on=("desc_wo_colour", "shipped_date"))
same_item_date

,orderid_x,customerid,sku,desc,desc_wo_colour,desc_colour,shipped,shipped_date,name,orderid_y,ordered_cd6,shipped_cd6,sku_cd6,desc_cd6,desc_colour_cd6
0,94040,8342,HOM1216,Manual Mixer (orange),Manual Mixer,(orange),2019-08-18 14:20:28,2019-08-18,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,HOM1216,Manual Mixer (orange),(orange)
1,94040,8342,HOM5127,Electric Toaster (blue),Electric Toaster,(blue),2019-08-18 14:20:28,2019-08-18,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,HOM5127,Electric Toaster (blue),(blue)
2,94026,4236,HOM5134,Electric Toaster (mauve),Electric Toaster,(mauve),2019-08-18 12:47:14,2019-08-18,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,HOM5127,Electric Toaster (blue),(blue)
3,154816,4274,COL3050,Noah's Jewelry (green),Noah's Jewelry,(green),2021-04-17 11:14:30,2021-04-17,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,COL4001,Noah's Jewelry (yellow),(yellow)
4,154828,2244,COL4001,Noah's Jewelry (yellow),Noah's Jewelry,(yellow),2021-04-17 12:22:55,2021-04-17,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,COL4001,Noah's Jewelry (yellow),(yellow)
5,154876,8342,COL4001,Noah's Jewelry (yellow),Noah's Jewelry,(yellow),2021-04-17 19:52:28,2021-04-17,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,COL4001,Noah's Jewelry (yellow),(yellow)
6,154856,8341,COL8453,Noah's Jewelry (blue),Noah's Jewelry,(blue),2021-04-17 16:30:00,2021-04-17,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,COL4001,Noah's Jewelry (yellow),(yellow)
7,200947,8342,COL3831,Noah's Jewelry (orange),Noah's Jewelry,(orange),2022-07-26 16:00:00,2022-07-26,Emily Randolph,200947,2022-07-26 08:18:05,2022-07-26 16:00:00,COL3831,Noah's Jewelry (orange),(orange)
8,200976,5911,COL9689,Noah's Jewelry (azure),Noah's Jewelry,(azure),2022-07-26 20:00:00,2022-07-26,Emily Randolph,200947,2022-07-26 08:18:05,2022-07-26 16:00:00,COL3831,Noah's Jewelry (orange),(orange)
9,154876,8342,HOM5838,Electric Crockpot (purple),Electric Crockpot,(purple),2021-04-17 19:52:28,2021-04-17,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,HOM5838,Electric Crockpot (purple),(purple)


In [41]:
same_item_date["diff_colour"] = same_item_date["desc_colour"]!=same_item_date["desc_colour_cd6"]
same_item_diff = same_item_date[same_item_date["diff_colour"]==True]
same_item_diff

,orderid_x,customerid,sku,desc,desc_wo_colour,desc_colour,shipped,shipped_date,name,orderid_y,ordered_cd6,shipped_cd6,sku_cd6,desc_cd6,desc_colour_cd6,diff_colour
2,94026,4236,HOM5134,Electric Toaster (mauve),Electric Toaster,(mauve),2019-08-18 12:47:14,2019-08-18,Emily Randolph,94040,2019-08-18 14:20:28,2019-08-18 14:20:28,HOM5127,Electric Toaster (blue),(blue),True
3,154816,4274,COL3050,Noah's Jewelry (green),Noah's Jewelry,(green),2021-04-17 11:14:30,2021-04-17,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,COL4001,Noah's Jewelry (yellow),(yellow),True
6,154856,8341,COL8453,Noah's Jewelry (blue),Noah's Jewelry,(blue),2021-04-17 16:30:00,2021-04-17,Emily Randolph,154876,2021-04-17 19:52:28,2021-04-17 19:52:28,COL4001,Noah's Jewelry (yellow),(yellow),True
8,200976,5911,COL9689,Noah's Jewelry (azure),Noah's Jewelry,(azure),2022-07-26 20:00:00,2022-07-26,Emily Randolph,200947,2022-07-26 08:18:05,2022-07-26 16:00:00,COL3831,Noah's Jewelry (orange),(orange),True
12,95558,6080,COL8758,Noah's Poster (mauve),Noah's Poster,(mauve),2019-09-09 17:00:00,2019-09-09,Emily Randolph,96220,2019-09-09 16:30:05,2019-09-09 17:45:00,COL4384,Noah's Poster (white),(white),True
13,95551,9605,COL9448,Noah's Poster (blue),Noah's Poster,(blue),2019-09-09 16:45:00,2019-09-09,Emily Randolph,96220,2019-09-09 16:30:05,2019-09-09 17:45:00,COL4384,Noah's Poster (white),(white),True
15,86372,8835,HOM7757,Electric Machine (purple),Electric Machine,(purple),2019-06-01 12:50:25,2019-06-01,Emily Randolph,86371,2019-06-01 12:50:16,2019-06-01 12:50:16,HOM5139,Electric Machine (amber),(amber),True


From the clue **"I was just about to leave when someone behind me said ‘Miss! You dropped something!’"**. Emily and the person who switch item with her still in Noah's shop. They must buy item in approximately same time. And there is only one order in the list that bought item about the same time as Emily.

In [42]:
customers[customers["customerid"]==8835]

,customerid,name,address,citystatezip,birthdate,phone,JD
7834,8835,Jonathan Adams,644 Targee St,"Staten Island, NY 10304",1975-08-26,315-618-5263,False


## Day 8 <a name="day8"></a>

From the clue **"Apparently he owns an entire set of Noah’s collectibles!"**. Let's find Noah's collectible list and then find the customer who has the most Noah's collectibles.

In [43]:
def noah_merch(desc):
    desc_list = desc.lower().split(" ")
    if "noah's" in desc_list:
        return True
    else:
        return False

In [44]:
product_noah = products.copy()
product_noah["noah"] = product_noah["desc"].apply(noah_merch)
product_noah = product_noah[product_noah["noah"]==True]
product_noah

,sku,desc,wholesale_cost,noah
9,TOY0085,Noah's Toy Soldier,11.96,True
30,COL0263,Noah's Jersey (orange),21.08,True
38,COL0295,Noah's Jersey (red),15.42,True
53,COL0464,Noah's Action Figure (magenta),23.93,True
68,COL0638,Noah's Jewelry (purple),43.98,True
...,...,...,...,...
1084,TOY9571,Noah's Stickers,1.13,True
1091,COL9688,Noah's Bobblehead (azure),11.42,True
1092,COL9689,Noah's Jewelry (azure),62.01,True
1109,TOY9830,Noah's Yoyo,5.63,True


In [45]:
noah_list = list(product_noah["sku"])
orders_items_noah = orders_items[orders_items["sku"].isin(noah_list)]
orders_items_noah = pd.merge(orders_items_noah, orders, how="inner", on="orderid")
orders_items_noah

,orderid,sku,qty,unit_price,customerid,ordered,shipped,items,total
0,1001,COL0820,1,25.52,4308,2017-01-31 00:32:19,2017-01-31 07:15:00,NaN,25.52
1,1012,COL0295,1,20.39,1799,2017-01-31 09:26:43,2017-01-31 09:26:43,NaN,26.68
2,1013,COL9689,1,74.09,6599,2017-01-31 09:28:19,2017-01-31 18:00:00,NaN,78.41
3,1017,TOY9830,1,7.34,8020,2017-01-31 10:00:07,2017-01-31 16:45:00,NaN,7.34
4,1023,COL4944,1,23.55,10971,2017-01-31 11:21:56,2017-01-31 11:21:56,NaN,23.55
...,...,...,...,...,...,...,...,...,...
36147,215159,COL8297,1,36.01,4827,2022-12-16 14:37:47,2022-12-16 14:37:47,NaN,36.01
36148,215164,COL8506,1,10.38,7498,2022-12-16 15:18:37,2022-12-16 16:00:00,NaN,15.47
36149,215200,COL8308,2,12.72,3643,2022-12-16 20:47:44,2022-12-23 13:45:00,NaN,25.44
36150,215201,COL3461,1,7.86,11468,2022-12-16 20:52:05,2022-12-21 17:30:00,NaN,15.86


Remove the duplicates, just in case someone buy one collectibles more than one time.

In [46]:
order_noah_uniq = orders_items_noah[["sku", "customerid"]].drop_duplicates()
order_noah_uniq

,sku,customerid
0,COL0820,4308
1,COL0295,1799
2,COL9689,6599
3,TOY9830,8020
4,COL4944,10971
...,...,...
36144,COL0732,4602
36146,COL8942,6548
36147,COL8297,4827
36149,COL8308,3643


In [47]:
order_noah_count = order_noah_uniq.groupby("customerid").count().reset_index().sort_values(by="sku", ascending=False)
order_noah_count.head()

,customerid,sku
1168,4308,84
700,3076,31
1531,5313,31
260,1797,28
80,1228,28


In [48]:
customers[customers["customerid"]==4308]

,customerid,name,address,citystatezip,birthdate,phone,JD
3307,4308,Travis Bartlett,2527B Adam Clayton Powell Jr Blvd,"Manhattan, NY 10039",1942-07-22,929-906-5980,False
10499,4308,Travis Bartlett,2527B Adam Clayton Powell Jr Blvd,"Manhattan, NY 10039",1942-07-22,929-906-5980,False
